# Evaluate Performance of model using Backtest Pipeline

In [22]:
import pandas as pd
import cufflinks
from openstf.pipeline.train_create_forecast_backtest import train_model_and_forecast_back_test

In [16]:
# define properties of training/prediction. We call this a 'prediction_job'
pj=dict(id=287,
        model='xgb',
        quantiles=[0.10,0.30,0.50,0.70,0.90],
        model_type_group=None, # Note, this should become optional
        hyper_params={}, # Note, this should become optional
        feature_names=None, # Note, this should become optional
        forecast_type="demand", # Note, this should become optional
        name=None,
        description=None,
       )

# Load input data
input_data = pd.read_csv('data/get_model_input_pid_287.csv', index_col='index', parse_dates=True)


In [17]:
# Perform the backtest
forecast, model, train_data, validation_data, test_data = train_model_and_forecast_back_test(
    pj,
    input_data,
    training_horizons=[0.25, 47.0],
 )

2021-08-25 11:07.48 [debug    ] Changed 194 values of constant load to NA. num_const_load_values=194
2021-08-25 11:07.48 [debug    ] Removed 194 NA values          num_removed_values=194
2021-08-25 11:07.52 [info     ] Postproces in preparation of storing


# Evaluate results

In [62]:
for horizon in set(forecast.horizon):
    fig = forecast.loc[forecast.horizon==0.25,['quantile_P10','quantile_P30',
                    'quantile_P50','quantile_P70','quantile_P90','realised','forecast']].iplot(asFigure=True,
                                                                                   title=f"Horizon: {horizon}")
    fig.update_traces(
         line=dict(color="green", width=1), fill='tonexty', fillcolor='rgba(0, 255, 0, 0.1)',
         selector=lambda x: 'quantile' in x.name and x.name != 'quantile_P10')
    fig.update_traces(
         line=dict(color="green", width=1),
         selector=lambda x: 'quantile_P10' == x.name)
    fig.update_traces(
         line=dict(color="red", width=2),
         selector=lambda x: 'realised' in x.name)
    fig.update_traces(
         line=dict(color="blue", width=2),
         selector=lambda x: 'forecast' in x.name)
    fig.show()

In [59]:
forecast['err']=forecast['realised']-forecast['forecast']
mea = forecast.pivot_table(index='horizon', values=['err'], aggfunc=lambda x: x.abs().mean())
mea.index=mea.index.astype(str)
mea.iplot(kind='bar',
          layout=dict(title='MAE',
                      xaxis=dict(title='horizon'),
                      yaxis=dict(title='MAE [MW]')))